In [18]:
!cp -r ../input/d/romanofrancesco/recsys-repo/* ./

In [19]:
%config Completer.use_jedi = False
import pandas as pd
import numpy as np
import scipy.sparse as sps
import matplotlib.pyplot as pyplot
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

In [20]:
#load data_train, matrix of users interaction
URM_path = "../input/recommender-system-2021-challenge-polimi/data_train.csv"
URM_all_dataframe = pd.read_csv(filepath_or_buffer=URM_path, 
                                sep=",",
                                dtype={0:int, 1:int, 2:float},
                                header=0)
URM_all_dataframe.columns = ["UserID", "ItemID", "Interaction"]

In [21]:
URM_all_dataframe.head()


,UserID,ItemID,Interaction
0,0,53,1.0
1,0,209,1.0
2,0,223,1.0
3,0,249,1.0
4,0,435,1.0


In [22]:
userID_unique = URM_all_dataframe["UserID"].unique()
itemID_unique = URM_all_dataframe["ItemID"].unique()

n_users = len(userID_unique)
n_items = len(itemID_unique)
n_interactions = len(URM_all_dataframe)

print ("Number of items\t {}, Number of users\t {}".format(n_items, n_users))
print ("Max ID items\t {}, Max Id users\t {}\n".format(max(itemID_unique), max(userID_unique)))
print ("Average interactions per user {:.2f}".format(n_interactions/n_users))
print ("Average interactions per item {:.2f}\n".format(n_interactions/n_items))

print ("Sparsity {:.2f} %".format((1-float(n_interactions)/(n_items*n_users))*100))

Number of items	 18059, Number of users	 13650
Max ID items	 18058, Max Id users	 13649

Average interactions per user 387.23
Average interactions per item 292.69

Sparsity 97.86 %


In [23]:
URM_all = sps.coo_matrix((URM_all_dataframe["Interaction"].values, 
                          (URM_all_dataframe["UserID"].values, URM_all_dataframe["ItemID"].values)))
URM_all = URM_all.tocsr() # to obtain fast access to rows (users)
URM_all

<13650x18059 sparse matrix of type '<class 'numpy.float64'>'
	with 5285664 stored elements in Compressed Sparse Row format>

In [26]:
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

# split data into train and validation data 80/20
URM_train, URM_valid = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.80)

In [28]:
from Evaluation.Evaluator import EvaluatorHoldout

#create an evaluator object to evaluate validation set
#we will use it for hyperparameter tuning
evaluator_valid = EvaluatorHoldout(URM_valid, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13644 ( 0.0%) Users that have less than 1 test interactions


In [31]:
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

#fit the puresvd model tuned in previous experiments
pureSVD = PureSVDRecommender(URM_train)
pureSVD.fit(num_factors=40)

PureSVDRecommender: Computing SVD decomposition...
PureSVDRecommender: Computing SVD decomposition... done in 2.94 sec


In [33]:
#check evaluation on pure svd
evaluator_valid.evaluateRecommender(pureSVD)

EvaluatorHoldout: Processed 13644 (100.0%) in 14.24 sec. Users per second: 958


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.376459                 0.377833  0.065929  0.228692    0.229247   
 
              MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                        ...   
 10      0.627092  0.390645  0.112207  0.964307      1.177908  ...   
 
        COVERAGE_ITEM_CORRECT COVERAGE_USER COVERAGE_USER_CORRECT  \
 cutoff                                                             
 10                  0.041752       0.99956              0.963883   
 
        DIVERSITY_GINI SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL  \
 cutoff                                                             
 10           0.011432        8.080757                   0.994442   
 
        RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY  \
 cutoff                                                

In [34]:
import numpy as np
import scipy.sparse as sps
from Recommenders.Recommender_utils import check_matrix
from sklearn.linear_model import ElasticNet
from Recommenders.BaseSimilarityMatrixRecommender import BaseItemSimilarityMatrixRecommender
from Utils.seconds_to_biggest_unit import seconds_to_biggest_unit
import time, sys
from tqdm import tqdm
from sklearn.utils._testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning

# os.environ["PYTHONWARNINGS"] = ('ignore::exceptions.ConvergenceWarning:sklearn.linear_model')
# os.environ["PYTHONWARNINGS"] = ('ignore:Objective did not converge:ConvergenceWarning:')

class SLIMElasticNetRecommender(BaseItemSimilarityMatrixRecommender):
    """
    Train a Sparse Linear Methods (SLIM) item similarity model.
    NOTE: ElasticNet solver is parallel, a single intance of SLIM_ElasticNet will
          make use of half the cores available
    See:
        Efficient Top-N Recommendation by Linear Regression,
        M. Levy and K. Jack, LSRS workshop at RecSys 2013.
        SLIM: Sparse linear methods for top-n recommender systems,
        X. Ning and G. Karypis, ICDM 2011.
        http://glaros.dtc.umn.edu/gkhome/fetch/papers/SLIM2011icdm.pdf
    """

    RECOMMENDER_NAME = "SLIMElasticNetRecommender"

    def __init__(self, URM_train, verbose = True):
        super(SLIMElasticNetRecommender, self).__init__(URM_train, verbose = verbose)

    @ignore_warnings(category=ConvergenceWarning)
    def fit(self, l1_ratio=0.1, alpha = 1.0, positive_only=True, topK = 100,**earlystopping_kwargs):

        assert l1_ratio>= 0 and l1_ratio<=1, "{}: l1_ratio must be between 0 and 1, provided value was {}".format(self.RECOMMENDER_NAME, l1_ratio)

        self.l1_ratio = l1_ratio
        self.positive_only = positive_only
        self.topK = topK


        # initialize the ElasticNet model
        self.model = ElasticNet(alpha=alpha,
                                l1_ratio=self.l1_ratio,
                                positive=self.positive_only,
                                fit_intercept=False,
                                copy_X=False,
                                precompute=True,
                                selection='random',
                                max_iter=100,
                                tol=1e-4)

        URM_train = check_matrix(self.URM_train, 'csc', dtype=np.float32)

        n_items = URM_train.shape[1]

        # Use array as it reduces memory requirements compared to lists
        dataBlock = 10000000

        rows = np.zeros(dataBlock, dtype=np.int32)
        cols = np.zeros(dataBlock, dtype=np.int32)
        values = np.zeros(dataBlock, dtype=np.float32)

        numCells = 0

        start_time = time.time()
        start_time_printBatch = start_time

        # fit each item's factors sequentially (not in parallel)
        for currentItem in range(n_items):

            # get the target column
            y = URM_train[:, currentItem].toarray()

            # set the j-th column of X to zero
            start_pos = URM_train.indptr[currentItem]
            end_pos = URM_train.indptr[currentItem + 1]

            current_item_data_backup = URM_train.data[start_pos: end_pos].copy()
            URM_train.data[start_pos: end_pos] = 0.0

            # fit one ElasticNet model per column
            self.model.fit(URM_train, y)

            # self.model.coef_ contains the coefficient of the ElasticNet model
            # let's keep only the non-zero values

            # Select topK values
            # Sorting is done in three steps. Faster then plain np.argsort for higher number of items
            # - Partition the data to extract the set of relevant items
            # - Sort only the relevant items
            # - Get the original item index

            nonzero_model_coef_index = self.model.sparse_coef_.indices
            nonzero_model_coef_value = self.model.sparse_coef_.data

            local_topK = min(len(nonzero_model_coef_value)-1, self.topK)

            relevant_items_partition = (-nonzero_model_coef_value).argpartition(local_topK)[0:local_topK]
            relevant_items_partition_sorting = np.argsort(-nonzero_model_coef_value[relevant_items_partition])
            ranking = relevant_items_partition[relevant_items_partition_sorting]

            for index in range(len(ranking)):

                if numCells == len(rows):
                    rows = np.concatenate((rows, np.zeros(dataBlock, dtype=np.int32)))
                    cols = np.concatenate((cols, np.zeros(dataBlock, dtype=np.int32)))
                    values = np.concatenate((values, np.zeros(dataBlock, dtype=np.float32)))


                rows[numCells] = nonzero_model_coef_index[ranking[index]]
                cols[numCells] = currentItem
                values[numCells] = nonzero_model_coef_value[ranking[index]]

                numCells += 1

            # finally, replace the original values of the j-th column
            URM_train.data[start_pos:end_pos] = current_item_data_backup

            elapsed_time = time.time() - start_time
            new_time_value, new_time_unit = seconds_to_biggest_unit(elapsed_time)


            if time.time() - start_time_printBatch > 300 or currentItem == n_items-1:
                self._print("Processed {} ({:4.1f}%) in {:.2f} {}. Items per second: {:.2f}".format(
                    currentItem+1,
                    100.0* float(currentItem+1)/n_items,
                    new_time_value,
                    new_time_unit,
                    float(currentItem)/elapsed_time))

                sys.stdout.flush()
                sys.stderr.flush()

                start_time_printBatch = time.time()

        # generate the sparse weight matrix
        self.W_sparse = sps.csr_matrix((values[:numCells], (rows[:numCells], cols[:numCells])),
                                       shape=(n_items, n_items), dtype=np.float32)

In [35]:
# create a Slim elasticnet object
slim_en = SLIMElasticNetRecommender(URM_train)
slim_en.fit(epochs = 500, l1_ratio = 0.0023170159712850467, alpha = 0.09078974149197175, positive_only = True, topK = 363)

SLIMElasticNetRecommender: Processed 1480 ( 8.2%) in 5.00 min. Items per second: 4.93
SLIMElasticNetRecommender: Processed 2972 (16.5%) in 10.00 min. Items per second: 4.95
SLIMElasticNetRecommender: Processed 4483 (24.8%) in 15.00 min. Items per second: 4.98
SLIMElasticNetRecommender: Processed 5997 (33.2%) in 20.00 min. Items per second: 5.00
SLIMElasticNetRecommender: Processed 7508 (41.6%) in 25.01 min. Items per second: 5.00
SLIMElasticNetRecommender: Processed 9028 (50.0%) in 30.01 min. Items per second: 5.01
SLIMElasticNetRecommender: Processed 10544 (58.4%) in 35.01 min. Items per second: 5.02
SLIMElasticNetRecommender: Processed 12054 (66.7%) in 40.01 min. Items per second: 5.02
SLIMElasticNetRecommender: Processed 13559 (75.1%) in 45.01 min. Items per second: 5.02
SLIMElasticNetRecommender: Processed 15076 (83.5%) in 50.02 min. Items per second: 5.02
SLIMElasticNetRecommender: Processed 16582 (91.8%) in 55.02 min. Items per second: 5.02
SLIMElasticNetRecommender: Processed 18

In [36]:
evaluator_valid.evaluateRecommender(slim_en)

EvaluatorHoldout: Processed 13644 (100.0%) in 27.35 sec. Users per second: 499


(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL       MAP MAP_MIN_DEN  \
 cutoff                                                                      
 10      0.399743                 0.401372  0.071387  0.247801    0.248569   
 
              MRR      NDCG       F1  HIT_RATE ARHR_ALL_HITS  ...  \
 cutoff                                                       ...   
 10      0.646522  0.413781  0.12114  0.972882      1.243795  ...   
 
        COVERAGE_ITEM_CORRECT COVERAGE_USER COVERAGE_USER_CORRECT  \
 cutoff                                                             
 10                   0.05255       0.99956              0.972454   
 
        DIVERSITY_GINI SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL  \
 cutoff                                                             
 10           0.010578         7.84531                     0.9918   
 
        RATIO_DIVERSITY_GINI RATIO_SHANNON_ENTROPY RATIO_AVERAGE_POPULARITY  \
 cutoff                                                   

In [72]:
# explore scores differences between the two models
user_id= 50
item_scores_slim_en = slim_en._compute_item_score(user_id)
item_scores_slim_en

array([[0.        , 0.00022715, 0.00142316, ..., 0.        , 0.        ,
        0.00227245]], dtype=float32)

In [73]:
item_scores_pureSVD = pureSVD._compute_item_score(user_id)
item_scores_pureSVD

array([-0.00170945, -0.00070905,  0.00303305, ..., -0.01095159,
        0.00171553, -0.01206828], dtype=float32)

In [ ]:
alpha = 0.7

new_item_scores = alpha * item_scores_slim_en + (1 - alpha) * item_scores_puresvd
new_item_scores

In [40]:
# create a recommender object which performs the hybridation of the 2 models
from Recommenders.BaseRecommender import BaseRecommender

class ScoresHybridRecommender(BaseRecommender):
    """ ScoresHybridRecommender
    Hybrid of two prediction scores R = R1*alpha + R2*(1-alpha)

    """

    RECOMMENDER_NAME = "ScoresHybridRecommender"

    def __init__(self, URM_train, recommender_1, recommender_2):
        super(ScoresHybridRecommender, self).__init__(URM_train)

        self.URM_train = sps.csr_matrix(URM_train)
        self.recommender_1 = recommender_1
        self.recommender_2 = recommender_2
        
        
    def fit(self, alpha = 0.5):
        self.alpha = alpha      


    def _compute_item_score(self, user_id_array, items_to_compute):
        
        # In a simple extension this could be a loop over a list of pretrained recommender objects
        item_weights_1 = self.recommender_1._compute_item_score(user_id_array)
        item_weights_2 = self.recommender_2._compute_item_score(user_id_array)

        item_weights = item_weights_1*self.alpha + item_weights_2*(1-self.alpha)

        return item_weights

In [69]:
# grid search over different values of hyperparameter alpha
alphas = [0.95280]

for alpha in alphas:
    scoreshybridrecommender = ScoresHybridRecommender(URM_train, slim_en, pureSVD)
    scoreshybridrecommender.fit(alpha = alpha)
    print("Hybrid model with alpha weight:")
    print(alpha)
    print(evaluator_valid.evaluateRecommender(scoreshybridrecommender))

Hybrid model with alpha weight:
0.9528
EvaluatorHoldout: Processed 13644 (100.0%) in 29.38 sec. Users per second: 464
(       PRECISION PRECISION_RECALL_MIN_DEN    RECALL      MAP MAP_MIN_DEN  \
cutoff                                                                     
10      0.400381                 0.401973  0.071421  0.24796    0.248716   

             MRR      NDCG        F1  HIT_RATE ARHR_ALL_HITS  ...  \
cutoff                                                        ...   
10      0.645173  0.413937  0.121218  0.972882      1.243069  ...   

       COVERAGE_ITEM_CORRECT COVERAGE_USER COVERAGE_USER_CORRECT  \
cutoff                                                             
10                  0.051276       0.99956              0.972454   

       DIVERSITY_GINI SHANNON_ENTROPY RATIO_DIVERSITY_HERFINDAHL  \
cutoff                                                             
10           0.010528        7.849191                   0.991931   

       RATIO_DIVERSITY_GINI RATIO_

# Create final recommendations

In [70]:
#let's use the hybrid model for final recommandations with the alpha tuned

recommender = ScoresHybridRecommender(URM_all, slim_en, pureSVD)
recommender.fit(alpha = 0.95280)
#evaluator_valid.evaluateRecommender(recommender)

In [75]:
test_users = pd.read_csv('../input/recommender-system-2021-challenge-polimi/data_target_users_test.csv')
test_users

,user_id
0,0
1,1
2,2
3,3
4,4
...,...
13645,13645
13646,13646
13647,13647
13648,13648


In [76]:
user_id = test_users['user_id']
recommendations = []
for user in user_id:
    recommendations.append(recommender.recommend(user,cutoff = 10))

In [77]:
for index in range(len(recommendations)):
    recommendations[index]=np.array(recommendations[index])
    
test_users['item_list']= recommendations
test_users['item_list'] = pd.DataFrame([str(line).strip('[').strip(']').replace("'","") for line in test_users['item_list']])
test_users.to_csv('submission.csv', index=False)